# Sprint Challenge
## *Data Science Sprint 13*

After a sprint of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). The data is sampled for you to something more manageable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge.

## Challenge Objectives
Successfully complete all these objectives to earn full credit.

**Successful completion is defined as passing all the unit tests in each objective.**  

There are 8 total possible points in this sprint challenge.


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on Yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernel

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu.

3) **Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section).**
____

### Part 0: Import Necessary Packages
For this section, you will need to import:
- `spacy`
- `Pandas`
- `Seaborn`
- `Matplotlib`
- `NearestNeighbors`
- `Pipeline`
- `TfidfVectorizer`
- `KneighborsClassifier`
- `GridSearchCV`
- `corpora`
- `LdaModel`
- `gensim`
- `re`

> **Note: This assignment is optimized to work with these specific packages. You can use import different packages, but note that this may affect how CodeGrade works, and may cause CodeGrade to fail.**

In [1]:
!python -m spacy download en_core_web_md

In [2]:
!pip install pyLDAvis==3.3.1

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
-m pip install --upgrade pip

In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import re

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
# Visible Testing
assert pd.__package__ == 'pandas'



### Part 0: Import Data

In [5]:


# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/bloominstituteoftechnology/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
# YOUR CODE HERE
df = pd.read_json(data_url, lines=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- Accept one document at a time
- Return a list of tokens

You are free to use any method you have learned this week.

**TO PASS CODEGRADE RUNTIME:**
- Do not run your tokenize function more than one time in your notebook! It is not needed until Part 4!

In [7]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.

nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
def tokenize(doc):
  """
  This function accepts a document and returns a list of tokens in the form of lemmas.
  It leverages the en_core_web_sm model in spaCy for tokenization
  """
  # Parse the document with spaCy
  doc = nlp(doc)

  # Create a list of tokens
  tokens = [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

  return tokens

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# Test the function
sample_doc = df.sample(n=1)["text"].iloc[0]
print(tokenize(sample_doc))

['move', 'March', '2013', 'leave', 'December', '2013', 'show', 'display', 'apartment', 'room', 'decent', 'tell', 'similar', 'design', 'style', 'eventually', 'come', 'realize', 'word', 'display', 'exactly', 'show', 'actual', 'apartment', 'move', 'day', '20', 'year', 'date', 'especially', 'bathroom', 'upset', 'neighbor', 'meet', 'move', 'nearly', 'time', 'type', 'studio', 'nearly', 'mirror', 'condition', 'display', 'apartment', 'water', 'white', 'try', 'inquire', 'office', 'tell', 'oh', 's', 'mesa', 'water', 'like', 'wait', 'clear', ' ', 'wait', 'nearly', '30', 'min', 'shower', 'hot', 'constant', 'roach', 'matter', 'clean', 'spray', 'speak', 'exterminator', 'tell', 'come', '2', 'week', 'come', 'twice', 'entire', 'span', '11', 'month', 'People', 'constantly', 'park', 'parking', 'spot', 'pay', 'extra', 'loud', 'neighbor', 'laughable', 'security', 'patrol', 'apartment', 'bad', 'month', 'stay', 'management', 'constantly', 'rude', 'speak', 'tenant', 'child', 'sarcasm', 'completely', 'unprofes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
    * Name that doc-term matrix `dtm`

In [10]:
%%time
# YOUR CODE HERE

# Initialize the Tfidfvectorizer
vect = TfidfVectorizer(tokenizer=tokenize, min_df=0.25, max_df=0.98, ngram_range=(1,2))

# Create a document-term matrix
dtm = vect.fit_transform(df['text'])

# Convert the dtm to a DataFrame for easier visualization
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 5min 18s, sys: 1.27 s, total: 5min 19s
Wall time: 6min



2. Write a fake review. Assign the text of the review to an object called `fake_review`.
3. Query the fake review for the 10 most similar reviews, print the text of the reviews.
    - Given the size of the dataset, use `NearestNeighbors` model for this. Name the model `nn`.

In [19]:
# Create and fit a NearestNeighbors model named "nn"
# YOUR CODE HERE

# Initialize the NearestNeighbors model
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

# Fit the model on the dtm
nn.fit(dtm)

# Write a fake review
fake_review = 'This product is amazing!! I love it and use it everyday'

# Transform the fake review into a dtm
fake_review_dtm = vect.transform([fake_review])

# Use the model to find the most similar reviews
distances, indices = nn.kneighbors(np.asarray(fake_review_dtm.todense()))

# Print the most similar reviews
for index in indices:
  print(df['text'].iloc[index])

219    I had an appointment at 10 a.m. for a simple o...
66     Fit for a bride helped me clean up my mothers ...
231    Closed   There's other guns there or you can w...
24     Their kids menu has Kraft Mac n Cheese. The pr...
109    The main course was just ok but that may have ...
254    Absolutely fantastic if you are looking for so...
229    Last Saturday I could not get a hold of my eld...
259    Very reasonable happy hour prices ($3,4,5) for...
203    This was the beat buffet I've been to in my en...
26     Just about every high end retailer has a store...
Name: text, dtype: object


In [16]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors._unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

In [20]:
# Visible Testing
assert isinstance(fake_review, str), "Did you write a review in the correct data type?"

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset.

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels).
    - Use that pipeline to predict a star rating for your fake review from Part 2.



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`.
    - Include 2 possible values for each parameter
        - **Keep the values for each parameter low. Extreme values will compromise runtime**
    - **Use `n_jobs` = 1**
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%%time
# Create a pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', RandomForestClassifier())
])

# Create parameter dictionary
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth': (5, 10)
}

# Initialize GridSearchCV
gs = GridSearchCV(pipeline, parameters, cv=5, n_jobs=1, verbose=1)

# Fit the GridSeachCV object to the data
gs.fit(df['text'], df['stars'])

# Predict a star rating for the fake review
fake_review_prediction = gs. predict(fake_review)
print('Predicted star rating for the fake review:', fake_review_prediction)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [ ]:
# Visible Testing
prediction = gs.predict(["This is your prediction statement."])[0]
assert prediction in df.stars.values, 'You gs object should be able to accept raw text within a list. Did you include a vectorizer in your pipeline?'

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations.
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this:

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.**

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz).

### 1. Estimate a LDA topic model of the review text

* Use the `tokenize` function you created earlier to create tokens.
* Create an `id2word` object.
> Hint: Use `corpora.Dictionary`
* Create a `corpus` object.
> Hint: Use `id2word.doc2bow`
* Instantiate an `lda` model.

>> Remember to read the LDA docs for more information on the various class attributes and methods available to you in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

In [ ]:
# Tokenize the reviews
df['tokens'] = df['text'].apply(tokenize)

# Create an id2word object
id2word = corpora.Dictionary(df['tokens'])

# Create a corpus object
corpus = [id2word.doc2bow(text) for text in df['tokens']]

# Define the number of topics
num_topics = 5

# Instantiate an LDA model
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

# Print the topics and teir most important words
lda.print_topics()

#### Testing

In [ ]:
# Visible Testing

assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

#### 2. Create 2 visualizations of the results:
1. Create a visualization using pyLDAvis. Run the cell, then comment out your code before submission, leaving the visualization in the cell.

2. Create a visualization using the matplotlib library and utilizing the subplots function. Assign this visualization to a variable called `visual_plot`.


In [ ]:
# Cell for pyLDAvis visualization
# YOUR CODE HERE
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, id2word)
vis

In [ ]:
# Cell for matplotlib visualzation
# YOUR CODE HERE
fig, ax = plt.subplots(figzise=(10,10))

for i in range(num_topics):
  x = [term[0] for term in lda.show_topic(i, 10)]
  y = [term[1] for term in lda.show_topic(i, 10)]

  ax.scatter(x, y, label=f'Topic {i+1}')
  for j, txt in enumerate(x):
    ax.annotate(txt, (x[j], y[j]))

ax.legend()
plt.show()

visual_plot = fig

In [ ]:
# Visible Testing
assert visual_plot.__module__ == 'matplotlib.axes._subplots', "You must create and assign to visual_plot a visualization"
